E01

In [1]:
words=open('names.txt').read().splitlines()

In [2]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
#building the trigram dictionary
t={}
for w in words:
    chs=['.']+['.'] + list(w) + ['.']
    for a,b,c in zip(chs,chs[1:],chs[2:]):
        trigram=(a+b,c)
        t[trigram]=t.get(trigram,0)+1
    

In [4]:
sorted(t.items(),key=lambda kv: -kv[1])

[(('..', 'a'), 4410),
 (('..', 'k'), 2963),
 (('..', 'm'), 2538),
 (('..', 'j'), 2422),
 (('..', 's'), 2055),
 (('ah', '.'), 1714),
 (('..', 'd'), 1690),
 (('na', '.'), 1673),
 (('..', 'r'), 1639),
 (('..', 'l'), 1572),
 (('..', 'c'), 1542),
 (('..', 'e'), 1531),
 (('an', '.'), 1509),
 (('on', '.'), 1503),
 (('.m', 'a'), 1453),
 (('..', 't'), 1308),
 (('..', 'b'), 1306),
 (('.j', 'a'), 1255),
 (('.k', 'a'), 1254),
 (('en', '.'), 1217),
 (('..', 'n'), 1146),
 (('ly', 'n'), 976),
 (('yn', '.'), 953),
 (('ar', 'i'), 950),
 (('..', 'z'), 929),
 (('ia', '.'), 903),
 (('..', 'h'), 874),
 (('ie', '.'), 858),
 (('an', 'n'), 825),
 (('el', 'l'), 822),
 (('an', 'a'), 804),
 (('ia', 'n'), 790),
 (('ma', 'r'), 776),
 (('in', '.'), 766),
 (('el', '.'), 727),
 (('ya', '.'), 716),
 (('an', 'i'), 703),
 (('.d', 'a'), 700),
 (('la', '.'), 684),
 (('er', '.'), 683),
 (('..', 'g'), 669),
 (('iy', 'a'), 669),
 (('la', 'n'), 647),
 (('.b', 'r'), 646),
 (('nn', 'a'), 633),
 (('.a', 'l'), 632),
 (('.c', 'a')

In [5]:
import torch


In [6]:
N=torch.zeros((27*27,27),dtype=torch.int32)

In [7]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}


In [8]:
for (a_b,c),count in t.items():
    i=stoi[a_b[0]]*27 + stoi[a_b[1]]
    j=stoi[c]
    N[i,j]+= count

In [9]:
N.shape

torch.Size([729, 27])

In [10]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [11]:
g=torch.Generator().manual_seed(2147483647)

In [12]:
P=(N+1).float()

In [13]:
P=P / P.sum(1,keepdim=True)

In [14]:
g=torch.Generator().manual_seed(2147483647)

for i in range(10):
    out=[]
    state = 0
    while True:
        p = P[state]
        c = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[c])
        if c == 0:
            break
        prev = state % 27
        state = prev * 27 + c
    print(''.join(out))

ce.
bra.
jalius.
rochityharlonimittain.
luwak.
ka.
da.
samiyah.
javer.
gotai.


In [15]:
log_likelihood = 0.0
n=0

for w in words:
    chs = ['.'] + ['.'] + list(w) + ['.']
    for a,b,c in zip(chs,chs[1:],chs[2:]):
        i = stoi[a]*27 + stoi[b]
        j = stoi[c]
        prob=P[i,j]
        log_prob=torch.log(prob)
        log_likelihood += log_prob
        n += 1
    
print(f'Average log likelihood: {log_likelihood/n}')
print(f'Negative log likelihood: {-log_likelihood/n}')

Average log likelihood: -2.2119739055633545
Negative log likelihood: 2.2119739055633545


E02

In [16]:
#building the bigram model
N = torch.zeros((27, 27), dtype=torch.int32)

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [17]:
P=(N+1).float()
P.sum(1,keepdim=True)
P=P/P.sum(1,keepdim=True)

In [18]:
g = torch.Generator().manual_seed(2147483647)


for i in range(20):
    out=[]
    ix=0
    while True:
        p=P[ix]
        #p=torch.ones(27)/27
        ix=torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix==0:
            break
    print(''.join(out))


cexze.
momasurailezitynn.
konimittain.
llayn.
ka.
da.
staiyaubrtthrigotai.
moliellavo.
ke.
teda.
ka.
emimmsade.
enkaviyny.
ftlspihinivenvorhlasu.
dsor.
br.
jol.
pen.
aisan.
ja.


In [19]:
#splitting data(80-10-10)
import random

random.seed(42)
words_shuffled = words[:]
random.shuffle(words_shuffled)

n = len(words_shuffled)
n_train = int(0.8 * n)
n_dev = int(0.9 * n)

words_train = words_shuffled[:n_train]
words_dev = words_shuffled[n_train:n_dev]
words_test = words_shuffled[n_dev:]

In [20]:
N_bigram = torch.zeros((27, 27), dtype=torch.int32)
for w in words_train:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N_bigram[ix1, ix2] += 1

In [21]:
P_bigram=(N_bigram+1).float()
P_bigram.sum(1,keepdim=True)
P_bigram=P_bigram/P_bigram.sum(1,keepdim=True)

In [22]:
def bigram_loss(words_split,P):
    loss=0.0
    n=0

    for w in words_split:
        chs = ['.'] + list(w) + ['.']
        for ch1, ch2 in zip(chs, chs[1:]):
            ix1 = stoi[ch1]
            ix2 = stoi[ch2]
            prob=P[ix1, ix2]
            log_prob=torch.log(prob)
            loss += -log_prob
            n += 1
    return loss/n

In [23]:
bg_train_loss = bigram_loss(words_train, P_bigram).item()
bg_dev_loss   = bigram_loss(words_dev,   P_bigram).item()
bg_test_loss  = bigram_loss(words_test,  P_bigram).item()

In [ ]:
N_trigram = torch.zeros((27*27, 27), dtype=torch.int32)

for w in words_train:
    chs=['.']+['.'] + list(w) + ['.']
    for a,b,c in zip(chs,chs[1:],chs[2:]):
        i=stoi[a]*27 + stoi[b]
        j=stoi[c]
        N_trigram[i,j] += 1

In [ ]:
P_trigram=(N_trigram+1).float()
P_trigram=P_trigram/P_trigram.sum(1,keepdim=True)

In [26]:
def trigram_loss(words_split,P):
    loss=0.0
    n=0

    for w in words_split:
        chs = ['.'] + ['.'] + list(w) + ['.']
        for a,b,c in zip(chs,chs[1:],chs[2:]):
            i = stoi[a]*27 + stoi[b]
            j = stoi[c]
            prob=P[i,j]
            log_prob=torch.log(prob)
            loss += -log_prob
            n += 1
    return loss/n

In [27]:
tg_train_loss = trigram_loss(words_train, P_trigram).item()
tg_dev_loss   = trigram_loss(words_dev,   P_trigram).item()
tg_test_loss  = trigram_loss(words_test,  P_trigram).item()

In [28]:
print(
    f"Bigram  - train: {bg_train_loss:.4f}, dev: {bg_dev_loss:.4f}, test: {bg_test_loss:.4f}"
)
print(
    f"Trigram - train: {tg_train_loss:.4f}, dev: {tg_dev_loss:.4f}, test: {tg_test_loss:.4f}"
)


Bigram  - train: 2.4544, dev: 2.4533, test: 2.4584
Trigram - train: 2.2157, dev: 2.2365, test: 2.2373


E03

In [29]:
def make_P_trigram(N,alpha):
    P=(N+alpha).float()
    P=P / P.sum(1,keepdim=True)
    return P

In [44]:
import numpy as np
alphas = np.arange(0.0, 0.2, 0.01)

results = []

for alpha in alphas:
    P = make_P_trigram(N_trigram, alpha)

    train_loss = trigram_loss(words_train, P).item()
    dev_loss   = trigram_loss(words_dev,   P).item()

    results.append((alpha, train_loss, dev_loss))

    print(f"alpha={alpha:<5} train={train_loss:.4f} dev={dev_loss:.4f}")


alpha=0.0   train=2.1830 dev=nan
alpha=0.01  train=2.1835 dev=2.2318
alpha=0.02  train=2.1840 dev=2.2282
alpha=0.03  train=2.1845 dev=2.2264
alpha=0.04  train=2.1850 dev=2.2252
alpha=0.05  train=2.1855 dev=2.2243
alpha=0.06  train=2.1859 dev=2.2237
alpha=0.07  train=2.1864 dev=2.2233
alpha=0.08  train=2.1869 dev=2.2229
alpha=0.09  train=2.1873 dev=2.2227
alpha=0.1   train=2.1877 dev=2.2225
alpha=0.11  train=2.1880 dev=2.2223
alpha=0.12  train=2.1884 dev=2.2222
alpha=0.13  train=2.1888 dev=2.2222
alpha=0.14  train=2.1892 dev=2.2221
alpha=0.15  train=2.1896 dev=2.2221
alpha=0.16  train=2.1900 dev=2.2221
alpha=0.17  train=2.1904 dev=2.2221
alpha=0.18  train=2.1907 dev=2.2221
alpha=0.19  train=2.1911 dev=2.2222


In [45]:
#skipping the first entry with alpha=0.0 to avoid choosing a model that has seen zero probabilities
best_alpha = min(results[1:], key=lambda x: x[2])[0]
print("Best alpha:", best_alpha)

Best alpha: 0.16


In [46]:
P_best= make_P_trigram(N_trigram, best_alpha)

tg_train_loss_best = trigram_loss(words_train, P_best).item()
tg_dev_loss_best   = trigram_loss(words_dev,   P_best).item()
tg_test_loss_best  = trigram_loss(words_test,  P_best).item()
print(
    f"Trigram with best alpha={best_alpha}:: " 
    f"train: {tg_train_loss_best:.4f}, dev: {tg_dev_loss_best:.4f}, test: {tg_test_loss_best:.4f}"
)

Trigram with best alpha=0.16:: train: 2.1900, dev: 2.2221, test: 2.2234


E04

In [49]:
N.shape

torch.Size([27, 27])

In [51]:
xs,ys=[],[]

for w in words_train:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [56]:
print(xs)
print(ys)

tensor([ 0, 25, 21,  ...,  4,  1, 14])
tensor([25, 21,  8,  ...,  1, 14,  0])


In [60]:
W=torch.randn((27,27),requires_grad=True)
logits=W[xs]
probs=torch.softmax(logits,dim=1)

In [58]:
torch.sum(probs,dim=1)

tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

In [62]:
for i in range(200):
    logits = W[xs]
    probs = torch.softmax(logits, dim=1)
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean()

    W.grad = None
    loss.backward()
    W.data -= 50 * W.grad

    if i % 20 == 0:
        print(i, loss.item())


0 3.973517417907715
20 2.566895008087158
40 2.5070011615753174
60 2.4878463745117188
80 2.478472948074341
100 2.472975730895996
120 2.4693968296051025
140 2.4669015407562256
160 2.465075969696045
180 2.4636895656585693


E05

In [65]:
W=torch.randn((27,27),requires_grad=True)

for i in range(200):
    logits = W[xs]
    loss = torch.nn.functional.cross_entropy(logits, ys)

    W.grad = None
    loss.backward()
    W.data -= 50 * W.grad

    if i % 20 == 0:
        print(i, loss.item())


0 3.8739466667175293
20 2.5743660926818848
40 2.511791467666626
60 2.490511655807495
80 2.480055332183838
100 2.4739725589752197
120 2.4700589179992676
140 2.467369556427002
160 2.465430974960327
180 2.4639780521392822
